In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data",one_hot=True)
#每个批次的大小，以矩阵放入
batch_size=100
#计算一共有多少个批次
n_batch=mnist.train.num_examples // batch_size

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:

#定义palcehoder
#None里面传入批次大小，将图像转为784向量
# [None, 784]是tensor的shape， None表示第一维(横向增长)是任意数量，784表示第二维是784维


#定义占位符
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])
keep_prob=tf.placeholder(tf.float32)
"""
定义学习率的变量
"""
lr=tf.Variable(0.1,dtype=tf.float32)

#创建网络
#建立一个神经元
W1=tf.Variable(tf.truncated_normal([784,1000],stddev=0.1))
b1=tf.Variable(tf.zeros([1000])+0.1)
L1=tf.nn.tanh(tf.matmul(x,W1)+b1)
L1_drop=tf.nn.dropout(L1,keep_prob)

W2=tf.Variable(tf.truncated_normal([1000,1000],stddev=0.1))
b2=tf.Variable(tf.zeros([1000])+0.1)
L2=tf.nn.tanh(tf.matmul(L1_drop,W2)+b2)
L2_drop=tf.nn.dropout(L2,keep_prob)


# W3=tf.Variable(tf.truncated_normal([1000,1000],stddev=0.1))
# b3=tf.Variable(tf.zeros([1000])+0.1)
# L3=tf.nn.tanh(tf.matmul(L2_drop,W3)+b3)
# L3_drop=tf.nn.dropout(L3,keep_prob)

W3=tf.Variable(tf.truncated_normal([1000,10],stddev=0.1))
b3=tf.Variable(tf.zeros([10])+0.1)
prediction=tf.nn.softmax(tf.matmul(L2_drop,W3)+b3)


loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#梯度下降
# train_step=tf.train.GradientDescentOptimizer(0.2).minimize(loss)

"""
这里在优化器的基础上动态修改学习率，学习率越来越小
"""
train_step=tf.train.AdadeltaOptimizer(lr).minimize(loss)

#继续初始化变量
init=tf.global_variables_initializer()

correct_prediction=tf.equal(tf.arg_max(y,1),tf.argmax(prediction,1))
#转化bool为float型
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#会话开始
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(50):
        #初始化lr,随着准确率上升，lr下降
        sess.run(tf.assign(lr,0.1*(0.95**epoch)))
        for batch in range(n_batch):
             
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        learning_rate=sess.run(lr)    
        test_acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
#         train_acc=sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:0.5})
        
        print("Iter"+str(epoch)+",Testing Accuracy"+str(test_acc)+"learning rate="+str(learning_rate))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.math.argmax` instead
Iter0,Testing Accuracy0.9158learning rate=0.1
Iter1,Testing Accuracy0.9335learning rate=0.095
Iter2,Testing Accuracy0.94learning rate=0.09025
Iter3,Testing Accuracy0.9448learning rate=0.0857375
Iter4,Testing Accuracy0.9486learning rate=0.081450626
Iter5,Testing Accuracy0.9509learning rate=0.077378094
